## Introduction

This project addresses inequality of educational opportunity in U.S. high schools. Here we will focus on average student performance on the ACT or SAT exams that students take as part of the college application process.

We expect a range of school performance on these exams, but is school performance predicted by socioeconomic factors?

## Import libraries

In [998]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey.

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high.

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


### Census data

The Census data is from the US Census Bureau's table [S1601](https://data.census.gov/table/ACSST1Y2022.S1601?t=Language%20Spoken%20at%20Home). This data set consists of information regarding language spoken at home categorized by ZIP Code Tabulation Areas. This is public information generated by the US federal government and is of high quality. This particular table is 2016's projected data.

## Load the data

Load the EdGap
 data set

In [999]:
edgap = pd.read_excel("https://github.com/SlimNaka/Education_Inequality/raw/main/EdGap_data.xlsx")

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [1000]:
!wget -O school_info.csv https://drive.usercontent.google.com/u/0/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP&export=download

--2024-04-26 19:05:04--  https://drive.usercontent.google.com/u/0/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.69.132, 2607:f8b0:4001:c08::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.69.132|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.usercontent.google.com/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP [following]
--2024-04-26 19:05:04--  https://drive.usercontent.google.com/uc?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP [following]
--2024-04-26 19:05:04--  https://drive.usercontent.google.com/download?id=1HvW2w-o2XZzCm4KTvnb1Bb3BvoAa14BP
Reusing existing connection to drive.usercontent.google.com:443.
HTTP request sent, awaiting 

In [1001]:
school_info = pd.read_csv('school_info.csv', encoding= 'unicode_escape', dtype={'MZIP': 'str'})

<ipython-input-1001-dc6eeef43c8f>:1: DtypeWarning: Columns (6,9,14,15,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('school_info.csv', encoding= 'unicode_escape', dtype={'MZIP': 'str'})


In [1002]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1003]:
path = "/content/drive/MyDrive/Colab Notebooks/Education_Inequality/The Census Data.csv"
census_df = pd.read_csv(path, low_memory=False)

## Explore the contents of the data sets

In [1004]:
edgap.head()
#CT = Census Tracked

,NCESSCH School ID,CT Unemployment Rate,CT Pct Adults with College Degree,CT Pct Childre In Married Couple Family,CT Median Household Income,School ACT average (or equivalent if SAT score),School Pct Free and Reduced Lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [1005]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7986 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NCESSCH School ID                                7986 non-null   int64  
 1   CT Unemployment Rate                             7972 non-null   float64
 2   CT Pct Adults with College Degree                7973 non-null   float64
 3   CT Pct Childre In Married Couple Family          7961 non-null   float64
 4   CT Median Household Income                       7966 non-null   float64
 5   School ACT average (or equivalent if SAT score)  7986 non-null   float64
 6   School Pct Free and Reduced Lunch                7986 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 436.9 KB


In [1006]:
school_info.head()

,SCHOOL_YEAR,FIPST,STATENAME,ST,SCH_NAME,LEA_NAME,STATE_AGENCY_NO,UNION,ST_LEAID,LEAID,...,G_10_OFFERED,G_11_OFFERED,G_12_OFFERED,G_13_OFFERED,G_UG_OFFERED,G_AE_OFFERED,GSLO,GSHI,LEVEL,IGOFFERED
0,2016-2017,1,ALABAMA,AL,Sequoyah Sch - Chalkville Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
1,2016-2017,1,ALABAMA,AL,Camps,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
2,2016-2017,1,ALABAMA,AL,Det Ctr,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
3,2016-2017,1,ALABAMA,AL,Wallace Sch - Mt Meigs Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported
4,2016-2017,1,ALABAMA,AL,McNeel Sch - Vacca Campus,Alabama Youth Services,1,NaN,AL-210,100002,...,Yes,Yes,Yes,No,No,No,07,12,High,As reported


In [1007]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102183 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102183 non-null  object 
 1   FIPST                102183 non-null  int64  
 2   STATENAME            102183 non-null  object 
 3   ST                   102183 non-null  object 
 4   SCH_NAME             102183 non-null  object 
 5   LEA_NAME             102183 non-null  object 
 6   STATE_AGENCY_NO      102183 non-null  object 
 7   UNION                2533 non-null    float64
 8   ST_LEAID             102183 non-null  object 
 9   LEAID                102183 non-null  object 
 10  ST_SCHID             102183 non-null  object 
 11  NCESSCH              102181 non-null  float64
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3         

This data set is much larger than the EdGap data set. Clearly the EdGap data set does not include all schools.

There are many null values in the school information data set, but we don't care about all of the columns.

In [1008]:
census_df.head()

,GEO_ID,NAME,S1601_C01_001E,S1601_C01_001M,S1601_C02_001E,S1601_C02_001M,S1601_C03_001E,S1601_C03_001M,S1601_C04_001E,S1601_C04_001M,...,S1601_C02_024M,S1601_C03_024E,S1601_C03_024M,S1601_C04_024E,S1601_C04_024M,S1601_C05_024E,S1601_C05_024M,S1601_C06_024E,S1601_C06_024M,Unnamed: 290
0,Geography,Geographic Area Name,Total!!Estimate!!Population 5 years and over,Total!!Margin of Error!!Population 5 years and...,Percent!!Estimate!!Population 5 years and over,Percent!!Margin of Error!!Population 5 years a...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Percen...,Percent of specified language speakers!!Percen...,...,Percent!!Margin of Error!!CITIZENS 18 YEARS AN...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Percen...,Percent of specified language speakers!!Percen...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Speak ...,Percent of specified language speakers!!Percen...,Percent of specified language speakers!!Percen...,NaN
1,8600000US00601,ZCTA5 00601,16863,248,(X),(X),2303,418,13.7,2.5,...,0.3,0,21,-,**,0,21,-,**,NaN
2,8600000US00602,ZCTA5 00602,37810,170,(X),(X),9187,746,24.3,2.0,...,0.1,0,28,-,**,0,28,-,**,NaN
3,8600000US00603,ZCTA5 00603,49018,774,(X),(X),12687,939,25.9,1.9,...,0.2,64,72,71.1,38.2,26,31,28.9,38.2,NaN
4,8600000US00606,ZCTA5 00606,5991,222,(X),(X),445,137,7.4,2.4,...,0.8,0,19,-,**,0,19,-,**,NaN


In [1009]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33121 entries, 0 to 33120
Columns: 291 entries, GEO_ID to Unnamed: 290
dtypes: float64(1), object(290)
memory usage: 73.5+ MB


All code related to the Census data will be executed in the section "Additional Step."

## Convert data types, if necessary

Thinking ahead, we will want to join the DataFrames using the identity of the school as the key. The identity is given by the NCESSCH school identity. This has a different name in the two DataFrames, and the value is an `int64` in the EdGap data set and a `float64` in the school information data set.

We will cast the `NCESSCH` column in the `school_info` DataFrame as an `int64`. We first need to drop rows where `NCESSCH` is a `NaN` value.

In [1010]:
school_info = school_info[school_info['NCESSCH'].isna() == False] # drops all NaN values from "NCESSCH" column
school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')

<ipython-input-1010-d37de4c0d664>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  school_info['NCESSCH'] = school_info['NCESSCH'].astype('int64')


In [1011]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102181 entries, 0 to 102182
Data columns (total 65 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   SCHOOL_YEAR          102181 non-null  object 
 1   FIPST                102181 non-null  int64  
 2   STATENAME            102181 non-null  object 
 3   ST                   102181 non-null  object 
 4   SCH_NAME             102181 non-null  object 
 5   LEA_NAME             102181 non-null  object 
 6   STATE_AGENCY_NO      102181 non-null  object 
 7   UNION                2531 non-null    float64
 8   ST_LEAID             102181 non-null  object 
 9   LEAID                102181 non-null  object 
 10  ST_SCHID             102181 non-null  object 
 11  NCESSCH              102181 non-null  int64  
 12  SCHID                102181 non-null  float64
 13  MSTREET1             102181 non-null  object 
 14  MSTREET2             1825 non-null    object 
 15  MSTREET3             2

## Are the data suitable for answering the question?

We want to perform quick exploratory data analysis to determine whether the data are sufficient to answer our question. If the data are not sufficient, we do not want to waste time doing anything that will not be productive.

We are interested in whether there is a relationship between the ACT score and the socioeconomic variables. We can make scatter plots to get a sense for the data.

### Renaming edgap columns

In [1012]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id",
              "CT Pct Adults with College Degree":"percent_college",
              "CT Unemployment Rate":"rate_unemployment",
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

###Make a pair plot to explore relationships between the variables.

In [1013]:
edgap.columns

Index(['id', 'rate_unemployment', 'percent_college', 'percent_married',
       'median_income', 'average_act', 'percent_lunch'],
      dtype='object')

In [1014]:
edgap = edgap.loc[edgap["average_act"]>5]
# removes outliers of very low ACT scores.

In [1015]:
# fig = plt.figure()
# sns.pairplot(edgap.drop(columns = 'id'))
# fig.set_size_inches(10,10)

## Select relevant subsets of the data

The school information data set contains a lot of information. We only need the year, school identity, location, and school type information.

Keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`.

In [1016]:
school_info = school_info[['SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL']]

### Renaming the school_info columns

Renaming the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL` to `year`, `id`, `state`, `zip_code`, `school_type`, `school_level`.

In [1017]:
school_info = school_info.rename(columns={'SCHOOL_YEAR':'year',
                                          'NCESSCH':'id',
                                          'MSTATE':'state',
                                          'MZIP':'zip_code',
                                          'SCH_TYPE_TEXT':'school_type',
                                          'LEVEL':'school_level'})

In [1018]:
edgap.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641


In [1019]:
school_info.head()

,year,id,state,zip_code,school_type,school_level
0,2016-2017,10000200277,AL,35220,Alternative School,High
1,2016-2017,10000201667,AL,36057,Alternative School,High
2,2016-2017,10000201670,AL,36057,Alternative School,High
3,2016-2017,10000201705,AL,36057,Alternative School,High
4,2016-2017,10000201706,AL,35206,Alternative School,High


In [1020]:
edgap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7983 entries, 0 to 7985
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7983 non-null   int64  
 1   rate_unemployment  7969 non-null   float64
 2   percent_college    7970 non-null   float64
 3   percent_married    7958 non-null   float64
 4   median_income      7963 non-null   float64
 5   average_act        7983 non-null   float64
 6   percent_lunch      7983 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 498.9 KB


In [1021]:
school_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102181 entries, 0 to 102182
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   year          102181 non-null  object
 1   id            102181 non-null  int64 
 2   state         102181 non-null  object
 3   zip_code      102181 non-null  object
 4   school_type   102179 non-null  object
 5   school_level  102179 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.5+ MB


## Join data frames

In [1022]:
df = pd.DataFrame(edgap.merge(school_info, how = 'left', on='id'))

In [1023]:
df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,year,state,zip_code,school_type,school_level
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,2016-2017,DE,19804,Regular School,High
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,2016-2017,DE,19709,Regular School,High
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,2016-2017,DE,19709,Regular School,High
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,2016-2017,DE,19958,Regular School,High
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,2016-2017,DE,19934,Regular School,High


In [1024]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7983 entries, 0 to 7982
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7983 non-null   int64  
 1   rate_unemployment  7969 non-null   float64
 2   percent_college    7970 non-null   float64
 3   percent_married    7958 non-null   float64
 4   median_income      7963 non-null   float64
 5   average_act        7983 non-null   float64
 6   percent_lunch      7983 non-null   float64
 7   year               7895 non-null   object 
 8   state              7895 non-null   object 
 9   zip_code           7895 non-null   object 
 10  school_type        7895 non-null   object 
 11  school_level       7895 non-null   object 
dtypes: float64(6), int64(1), object(5)
memory usage: 748.5+ KB


## Quality Control

Check for out-of-range values or values that do not match what we want to analyze. Either set values to `NaN` or remove the observations, as appropriate.

Dropping unneeded column: "year" and "school_level" types other than high school.

In [1025]:
df["year"].unique()
# Only one year. No need for it.

array(['2016-2017', nan], dtype=object)

In [1026]:
df["school_level"].unique()
# We are only looking at the high school level.

array(['High', 'Other', 'Not reported', nan, 'Elementary'], dtype=object)

In [1027]:
df = df.drop(columns = 'year')
df = df.loc[df['school_level'] == 'High']

Checking for impossible values.

In [1028]:
print("ru-,", sum(df["rate_unemployment"] < 0))
print("ru+,", sum(df["rate_unemployment"] > 1))
print("pc-,", sum(df["percent_college"] < 0))
print("pc+,", sum(df["percent_college"] > 1))
print("pm-,", sum(df["percent_married"] < 0))
print("pm+,", sum(df["percent_married"] > 1))
print("mi-,", sum(df["median_income"] < 0))
print("act-,", sum(df["average_act"] < 0))
print("act+,", sum(df["average_act"] > 36))
print("pl-,", sum(df["percent_lunch"] < 0))
print("pl+,", sum(df["percent_lunch"] > 1))

ru-, 0
ru+, 0
pc-, 0
pc+, 0
pm-, 0
pm+, 0
mi-, 0
act-, 0
act+, 0
pl-, 20
pl+, 0


Removing entries with impossible values.

In [1029]:
# From above, only percent_lunch needs correcting.
df = df[df['percent_lunch'] > 0]

In [1030]:
# Just in case, I want to drop any duplicate dates.
df = df.drop_duplicates()

## Additional step

We will look at how an area area being multilingual affects ACT scores.

We will prepare the data similar to the ed_gap and school_info data.

In [1031]:
census_column_names = list(census_df.columns.values)

In [1032]:
new_header = census_df.iloc[0]
census_df = census_df[1:]
census_df.columns = new_header
census_df = census_df.reset_index(drop=True)

In [1033]:
census_df.head()

,Geography,Geographic Area Name,Total!!Estimate!!Population 5 years and over,Total!!Margin of Error!!Population 5 years and over,Percent!!Estimate!!Population 5 years and over,Percent!!Margin of Error!!Population 5 years and over,"Percent of specified language speakers!!Speak English only or speak English very well""!!Estimate!!Population 5 years and over""","Percent of specified language speakers!!Speak English only or speak English very well""!!Margin of Error!!Population 5 years and over""","Percent of specified language speakers!!Percent speak English only or speak English very well""!!Estimate!!Population 5 years and over""","Percent of specified language speakers!!Percent speak English only or speak English very well""!!Margin of Error!!Population 5 years and over""",...,Percent!!Margin of Error!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages,"Percent of specified language speakers!!Speak English only or speak English very well""!!Estimate!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Speak English only or speak English very well""!!Margin of Error!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Percent speak English only or speak English very well""!!Estimate!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Percent speak English only or speak English very well""!!Margin of Error!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Speak English less than very well""!!Estimate!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Speak English less than very well""!!Margin of Error!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Percent speak English less than very well""!!Estimate!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""","Percent of specified language speakers!!Percent speak English less than very well""!!Margin of Error!!CITIZENS 18 YEARS AND OVER!!All citizens 18 years old and over!!Speak a language other than English!!Other languages""",NaN
0,8600000US00601,ZCTA5 00601,16863,248,(X),(X),2303,418,13.7,2.5,...,0.3,0,21,-,**,0,21,-,**,NaN
1,8600000US00602,ZCTA5 00602,37810,170,(X),(X),9187,746,24.3,2.0,...,0.1,0,28,-,**,0,28,-,**,NaN
2,8600000US00603,ZCTA5 00603,49018,774,(X),(X),12687,939,25.9,1.9,...,0.2,64,72,71.1,38.2,26,31,28.9,38.2,NaN
3,8600000US00606,ZCTA5 00606,5991,222,(X),(X),445,137,7.4,2.4,...,0.8,0,19,-,**,0,19,-,**,NaN
4,8600000US00610,ZCTA5 00610,26631,197,(X),(X),4449,555,16.7,2.1,...,0.2,0,25,-,**,0,25,-,**,NaN


In [1034]:
census_df["Geographic Area Name"] = census_df["Geographic Area Name"].astype(str)


In [1035]:
list(census_df.columns.values)

['Geography',
 'Geographic Area Name',
 'Total!!Estimate!!Population 5 years and over',
 'Total!!Margin of Error!!Population 5 years and over',
 'Percent!!Estimate!!Population 5 years and over',
 'Percent!!Margin of Error!!Population 5 years and over',
 'Percent of specified language speakers!!Speak English only or speak English very well"!!Estimate!!Population 5 years and over"',
 'Percent of specified language speakers!!Speak English only or speak English very well"!!Margin of Error!!Population 5 years and over"',
 'Percent of specified language speakers!!Percent speak English only or speak English very well"!!Estimate!!Population 5 years and over"',
 'Percent of specified language speakers!!Percent speak English only or speak English very well"!!Margin of Error!!Population 5 years and over"',
 'Percent of specified language speakers!!Speak English  less than very well"!!Estimate!!Population 5 years and over"',
 'Percent of specified language speakers!!Speak English  less than very w

We will take and rename the columns that are relevant to general multilingualism.

In [1036]:
census_df = census_df[["Geographic Area Name", "Total!!Estimate!!Population 5 years and over",
                       "Total!!Estimate!!Speak only English","Total!!Estimate!!Speak a language other than English",
                       'Percent of specified language speakers!!Speak English  less than very well"!!Estimate!!Population 5 years and over"']]

census_df = census_df.rename(columns={"Geographic Area Name":"zip_code",
              "Total!!Estimate!!Population 5 years and over":"persons_over_five",
              "Total!!Estimate!!Speak only English":"english_only",
              "Total!!Estimate!!Speak a language other than English":"english_other",
              'Percent of specified language speakers!!Speak English  less than very well"!!Estimate!!Population 5 years and over"':"not_english"})

In [1037]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33120 entries, 0 to 33119
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   zip_code           33120 non-null  object
 1   persons_over_five  33120 non-null  object
 2   english_only       33120 non-null  object
 3   english_other      33120 non-null  object
 4   not_english        33120 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


In [1038]:
# Code below removes the ZCTA5 prefix for the zip codes to match our main df.
census_zips = []
for i in census_df["zip_code"]:
  i = i.replace("ZCTA5 ","")
  census_zips.append(i)
census_df["zip_code"] = census_zips

In [1039]:
# Inner merge to focus on shared zips.
new_df = pd.DataFrame(df.merge(census_df, how = 'inner', on='zip_code'))

In [1040]:
new_df.head()

,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch,state,zip_code,school_type,school_level,persons_over_five,english_only,english_other,not_english
0,100001600143,0.117962,0.445283,0.346495,42820.0,20.433455,0.066901,DE,19804,Regular School,High,16498,14003,2495,1005
1,100008000024,0.063984,0.662765,0.767619,89320.0,19.498168,0.112412,DE,19709,Regular School,High,36112,33041,3071,750
2,100008000225,0.056460,0.701864,0.713090,84140.0,19.554335,0.096816,DE,19709,Regular School,High,36112,33041,3071,750
3,100017000029,0.044739,0.692062,0.641283,56500.0,17.737485,0.296960,DE,19958,Regular School,High,22808,21247,1561,487
4,100018000040,0.077014,0.640060,0.834402,54015.0,18.245421,0.262641,DE,19934,Regular School,High,13099,12022,1077,363


In [1041]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7100 non-null   int64  
 1   rate_unemployment  7088 non-null   float64
 2   percent_college    7089 non-null   float64
 3   percent_married    7081 non-null   float64
 4   median_income      7084 non-null   float64
 5   average_act        7100 non-null   float64
 6   percent_lunch      7100 non-null   float64
 7   state              7100 non-null   object 
 8   zip_code           7100 non-null   object 
 9   school_type        7100 non-null   object 
 10  school_level       7100 non-null   object 
 11  persons_over_five  7100 non-null   object 
 12  english_only       7100 non-null   object 
 13  english_other      7100 non-null   object 
 14  not_english        7100 non-null   object 
dtypes: float64(6), int64(1), object(8)
memory usage: 832.2+ KB


Changing the numbers into actual numbers so we can calculate how many people speak English fluently as a second language.

In [1042]:
new_df["persons_over_five"] = new_df["persons_over_five"].astype('int64')
new_df["english_only"] = new_df["english_only"].astype('int64')
new_df["english_other"] = new_df["english_other"].astype('int64')
new_df["not_english"] = new_df["not_english"].astype('int64')
esl = []
for i in (new_df["english_other"] - new_df["not_english"]):
  esl.append(i)

new_df["esl"] = esl

Calculating our main variable, percentage of an area that is multilingual, and adding it to our dataframe.

In [1043]:
percent_multiling = []
for i in (new_df["esl"] / new_df["persons_over_five"]):
  percent_multiling.append(i)

new_df["percent_multiling"] = percent_multiling

In [1044]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7100 non-null   int64  
 1   rate_unemployment  7088 non-null   float64
 2   percent_college    7089 non-null   float64
 3   percent_married    7081 non-null   float64
 4   median_income      7084 non-null   float64
 5   average_act        7100 non-null   float64
 6   percent_lunch      7100 non-null   float64
 7   state              7100 non-null   object 
 8   zip_code           7100 non-null   object 
 9   school_type        7100 non-null   object 
 10  school_level       7100 non-null   object 
 11  persons_over_five  7100 non-null   int64  
 12  english_only       7100 non-null   int64  
 13  english_other      7100 non-null   int64  
 14  not_english        7100 non-null   int64  
 15  esl                7100 non-null   int64  
 16  percent_multiling  7098 

Removing unnecessary columns.

In [1045]:
final_features = ["id", "rate_unemployment", "percent_college", "percent_married", "median_income",
                  "average_act", "percent_lunch", "percent_multiling", "state", "zip_code",
                  "school_type", "school_level"]
new_df = new_df[final_features]

In [1046]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7100 non-null   int64  
 1   rate_unemployment  7088 non-null   float64
 2   percent_college    7089 non-null   float64
 3   percent_married    7081 non-null   float64
 4   median_income      7084 non-null   float64
 5   average_act        7100 non-null   float64
 6   percent_lunch      7100 non-null   float64
 7   percent_multiling  7098 non-null   float64
 8   state              7100 non-null   object 
 9   zip_code           7100 non-null   object 
 10  school_type        7100 non-null   object 
 11  school_level       7100 non-null   object 
dtypes: float64(7), int64(1), object(4)
memory usage: 665.8+ KB


## Identify missing values

Determine whether there are missing values in the data set. Only identify them at this point; we will deal with them after creating training and testing splits of the data set.

In [1047]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7100 non-null   int64  
 1   rate_unemployment  7088 non-null   float64
 2   percent_college    7089 non-null   float64
 3   percent_married    7081 non-null   float64
 4   median_income      7084 non-null   float64
 5   average_act        7100 non-null   float64
 6   percent_lunch      7100 non-null   float64
 7   percent_multiling  7098 non-null   float64
 8   state              7100 non-null   object 
 9   zip_code           7100 non-null   object 
 10  school_type        7100 non-null   object 
 11  school_level       7100 non-null   object 
dtypes: float64(7), int64(1), object(4)
memory usage: 665.8+ KB


This shows us that:

rate_unemployment  has 12 NaN values

percent_college has 11 NaN values

percent_married has 19 NaN values

median_income has 16 NaN values

percent_multiling has 2 NaN values

## Train test split

We will use the test set approach to compare models predicting `average_act`.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/ML_dataset_training_validation_test_sets.png/800px-ML_dataset_training_validation_test_sets.png)

Split the data into training and testing sets. Keep 20% of the data for the test set.

In [1048]:
features = ["id", "rate_unemployment", "percent_college", "percent_married", "median_income",
            "percent_lunch", "percent_multiling", "state", "zip_code", "school_type", "school_level"]

target = ["average_act"]

X = new_df[features]
y = new_df[target]

In [1049]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100 entries, 0 to 7099
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 7100 non-null   int64  
 1   rate_unemployment  7088 non-null   float64
 2   percent_college    7089 non-null   float64
 3   percent_married    7081 non-null   float64
 4   median_income      7084 non-null   float64
 5   average_act        7100 non-null   float64
 6   percent_lunch      7100 non-null   float64
 7   percent_multiling  7098 non-null   float64
 8   state              7100 non-null   object 
 9   zip_code           7100 non-null   object 
 10  school_type        7100 non-null   object 
 11  school_level       7100 non-null   object 
dtypes: float64(7), int64(1), object(4)
memory usage: 665.8+ KB


In [1050]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5680, 11)
(1420, 11)
(5680, 1)
(1420, 1)


In [1051]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5680 entries, 3248 to 860
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5680 non-null   int64  
 1   rate_unemployment  5673 non-null   float64
 2   percent_college    5674 non-null   float64
 3   percent_married    5667 non-null   float64
 4   median_income      5670 non-null   float64
 5   percent_lunch      5680 non-null   float64
 6   percent_multiling  5680 non-null   float64
 7   state              5680 non-null   object 
 8   zip_code           5680 non-null   object 
 9   school_type        5680 non-null   object 
 10  school_level       5680 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 532.5+ KB


In [1052]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 4977 to 4818
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1420 non-null   int64  
 1   rate_unemployment  1415 non-null   float64
 2   percent_college    1415 non-null   float64
 3   percent_married    1414 non-null   float64
 4   median_income      1414 non-null   float64
 5   percent_lunch      1420 non-null   float64
 6   percent_multiling  1418 non-null   float64
 7   state              1420 non-null   object 
 8   zip_code           1420 non-null   object 
 9   school_type        1420 non-null   object 
 10  school_level       1420 non-null   object 
dtypes: float64(6), int64(1), object(4)
memory usage: 133.1+ KB


## Normalizing

We will normalize the features for both the train and test sets, but leave the target as is.

In [1053]:
ids_train = X_train["id"]   #   We want to separate the ids from being
ids_test = X_test["id"]     #   normalized and then add it back later.

X_train = X_train.drop(columns = "id")
X_test = X_test.drop(columns = "id")

In [1054]:
mean = X_train.select_dtypes(include = "number").mean()
std = X_train.select_dtypes(include = "number").std()

X_train_norm = (X_train - mean)/std
X_test_norm = (X_test - mean)/std

In [1055]:
print(X_train_norm.info())
print()
print(X_test_norm.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5680 entries, 3248 to 860
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   median_income      5670 non-null   float64
 1   percent_college    5674 non-null   float64
 2   percent_lunch      5680 non-null   float64
 3   percent_married    5667 non-null   float64
 4   percent_multiling  5680 non-null   float64
 5   rate_unemployment  5673 non-null   float64
 6   school_level       0 non-null      object 
 7   school_type        0 non-null      object 
 8   state              0 non-null      object 
 9   zip_code           0 non-null      object 
dtypes: float64(6), object(4)
memory usage: 488.1+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 4977 to 4818
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   median_income      1414 non-null   float64
 1   percen

Our normalized train and test sets now have NaNs for their oject-type values. We will fix this below.

## Data imputation

Use an imputation method to replace missing values in the columns corresponding to predictor variables in the analysis.

In [1056]:
num_vars = ["rate_unemployment", "percent_college", "percent_married", "median_income", "percent_lunch", "percent_multiling"]
imp = IterativeImputer(random_state=0)
X_train_norm[num_vars] = imp.fit_transform(X_train[num_vars])

The code above fills in the missing data for our numbered variables.

The code below is taking the newly normalized (and imputated for X_train) values and recasting them into the X_train and X_test dataframes.

In [1057]:
X_train["rate_unemployment"] = X_train_norm["rate_unemployment"]
X_train["percent_college"] = X_train_norm["percent_college"]
X_train["percent_married"] = X_train_norm["percent_married"]
X_train["median_income"] = X_train_norm["median_income"]
X_train["percent_lunch"] = X_train_norm["percent_lunch"]
X_train["percent_multiling"] = X_train_norm["percent_multiling"]
X_train.insert(loc=0, column='id', value=ids_train) #Let's replace the IDs while we are at it.
#X_train["id"] = ids_train

X_test["rate_unemployment"] = X_test_norm["rate_unemployment"]
X_test["percent_college"] = X_test_norm["percent_college"]
X_test["percent_married"] = X_test_norm["percent_married"]
X_test["median_income"] = X_test_norm["median_income"]
X_test["percent_lunch"] = X_test_norm["percent_lunch"]
X_test["percent_multiling"] = X_test_norm["percent_multiling"]
X_test.insert(loc=0, column='id', value=ids_test) #Let's replace the IDs while we are at it.
#X_test["id"] = ids_test

Now we're sliding those untouched y dataframes back into their respective "train" and "test" dataframes.

In [1058]:
X_train["average_act"] = y_train
X_test["average_act"] = y_test

print(X_train.info())
print()
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5680 entries, 3248 to 860
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5680 non-null   int64  
 1   rate_unemployment  5680 non-null   float64
 2   percent_college    5680 non-null   float64
 3   percent_married    5680 non-null   float64
 4   median_income      5680 non-null   float64
 5   percent_lunch      5680 non-null   float64
 6   percent_multiling  5680 non-null   float64
 7   state              5680 non-null   object 
 8   zip_code           5680 non-null   object 
 9   school_type        5680 non-null   object 
 10  school_level       5680 non-null   object 
 11  average_act        5680 non-null   float64
dtypes: float64(7), int64(1), object(4)
memory usage: 576.9+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 4977 to 4818
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
-

## Create relevant derived variables as new columns

We may already know that we want to process the data to create new variables from the existing variables. However, we often start analyzing the data and realize that it is useful to create new variables derived from the existing variables. Or, we might not create any new columns. It is fine to return to modify this step after exploring the data further.

## Export the clean .csv files

Export files with the training and testing data.

In [1059]:
X_train.info() # one last check

<class 'pandas.core.frame.DataFrame'>
Index: 5680 entries, 3248 to 860
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5680 non-null   int64  
 1   rate_unemployment  5680 non-null   float64
 2   percent_college    5680 non-null   float64
 3   percent_married    5680 non-null   float64
 4   median_income      5680 non-null   float64
 5   percent_lunch      5680 non-null   float64
 6   percent_multiling  5680 non-null   float64
 7   state              5680 non-null   object 
 8   zip_code           5680 non-null   object 
 9   school_type        5680 non-null   object 
 10  school_level       5680 non-null   object 
 11  average_act        5680 non-null   float64
dtypes: float64(7), int64(1), object(4)
memory usage: 576.9+ KB


In [1060]:
X_test.info() # one last check

<class 'pandas.core.frame.DataFrame'>
Index: 1420 entries, 4977 to 4818
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1420 non-null   int64  
 1   rate_unemployment  1415 non-null   float64
 2   percent_college    1415 non-null   float64
 3   percent_married    1414 non-null   float64
 4   median_income      1414 non-null   float64
 5   percent_lunch      1420 non-null   float64
 6   percent_multiling  1418 non-null   float64
 7   state              1420 non-null   object 
 8   zip_code           1420 non-null   object 
 9   school_type        1420 non-null   object 
 10  school_level       1420 non-null   object 
 11  average_act        1420 non-null   float64
dtypes: float64(7), int64(1), object(4)
memory usage: 144.2+ KB


In [1061]:
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)